In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
import numpy as np

min_max_scaler = preprocessing.MinMaxScaler()

from collections import Counter


In [2]:
def compare_and_score(sol, pred):
    numerator = len(list((Counter(sol) & Counter(pred)).elements()))
    recall = numerator / len(sol)
    return (recall,numerator,len(sol)-numerator)


In [3]:
file_name = "mapping_nbest.txt"
utt_list = []
utt_dict = {}
with open(file_name, 'r', encoding='utf-8',  errors='ignore') as f:
    for line in f:
        word = line.split("\n")[0].split(",")
        utt_list.append(word[0])
        utt_dict[word[0]] = word[2]

In [10]:
recalls = []
j = 0
ptrue_positive = 0
pfalse_negative = 0
for i in range(500):
    tf = pd.read_csv("ground_truth_csv/"+str(i)+".csv",sep=";")
    tf["tuple"] = tf["lemma"]+";"+tf["morphology"]
    gt_list = tf["tuple"].values.tolist()
    flag_ = 0
    pred_list = []
    count = 0
    for k in range(10):
        count += 1
        if j+k >= 4732:
            break
        utt = utt_list[j+k]
        utt_grp = utt_dict[utt]
        if k == 0:
            pre_utt_grp = utt_grp
        if pre_utt_grp != utt_grp:
            j = j + count - 1
            flag_ = 1
            break
        pf = pd.read_csv("nbest_csv/"+str(j+k)+".csv",sep=",")
        pf["tuple"] = pf["lemma"]+";"+pf["morph"]
        pd_list = pf["tuple"].values.tolist()
        pred_list += pd_list    
    if flag_ == 0:
        j = j + count
    recall,true_positive,false_negative = compare_and_score(gt_list,pred_list)
    #print(gt_list)
    #print(pred_list)
    #print(recall)
    recalls.append(recall)
    ptrue_positive += true_positive
    pfalse_negative += false_negative
avg_recall = np.mean(recalls)
print(avg_recall)
print(ptrue_positive/(ptrue_positive+pfalse_negative))

0.8777442361216355
0.8804318488529015
